# 1.0 - Introdução

O documento presente tem por objetivo viabilizar a criação de uma solução capaz de analisar, filtrar e classificar dados de pacientes com câncer a fim de identificar a variabilidade da evolução da doença, bem como as possíveis respostas a tratamentos já implantados, para, assim, detectar padrões que indiquem aos profissionais de saúde uma possível estimativa de risco e grau de sobrevida de um paciente.

## 1.1 - Configurações iniciais

Importando as bibliotecas necessárias à manipulação do banco de dados:

In [ ]:
# Importando as bibliotecas necessárias à manipulação do banco de dados.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Definindo a quantidade máxima de colunas como indeterminada.
pd.set_option('display.max_columns', None)

Para que este notebook funcione corretamente, é necessário, primeiramente, fazer o upload do banco de dados nos arquivos do caderno. Observação: atente-se para que o arquivo esteja no formado .XLSX (EXCEL). Assim que fizer isso, digite o nome do arquivo na célula abaixo e a execute.

In [ ]:
#@title Caminho do arquivo de banco de dados

# Lendo o banco de dados a partir da planilha recebida.

file_path = "./BDIP.xlsx" #@param {type:"string"}"

table = pd.read_excel(file_path)

A fim de eliminar, logo de início, todas as colunas que não possuem ao menos um dado, executa-se:

In [ ]:
# Limpando todas as colunas que possuem todos os seus valores vazios.
table.dropna(how='all', axis=1, inplace=True)

E, com objetivo de validar se tudo ocorreu bem, exibe-se a tabela abaixo:

In [ ]:
# Imprimindo os quatro primeiros registros da tabela.
table.head()

,record_id,redcap_repeat_instrument,redcap_repeat_instance,dob,escolari,level_education,race,pregnancy_history,partos,abortion,breast_feeding,breast_feeding_time,menarche,period,bmi,contraceptive_method___1,contraceptive_method___2,contraceptive_method___3,contraceptive_method___4,contraceptive_method___999,contraceptive,hormone_therapy,hormone_therapy_tasy,tempo_rep_hormo,tempo_rep_hormo_tasy,antec_fam_cancer_mama,tobaco,alcohol,tobaco_type,alcohol_type___1,alcohol_type___2,alcohol_type___3,family_cancer,familial_degree___1,familial_degree___2,familial_degree___3,family_cancer_id___1,family_cancer_id___2,family_cancer_id___3,family_cancer_id___4,family_cancer_id___5,family_cancer_id___6,mother_tumor,sister_n,daughter_n,sister_tumor_1,daughter_tumor_1,birads,benign,primary_diganosis,diff_tubular,nuclear_grade,mitotic_index,histological_grade,grau_hist,multicentricidade,multifocal,tumor_margin,er_ihc,pr_ihc,tumor_subtype,progesteron_perct,er_percentage,her2_ihc,her2_fish,ki67_perc,date_last_fu,follow_up_days,ultinfo,recidive,dtrecidiva,follow_up_days_recidive,reclocal,recregio,recdist,rec01,rec02,rec03,rec04,trathosp,topo,basediag,desctopo,morfo,descmorfo,tumor_stage,t,n,m,meta01,meta02,meta03,meta04,treatment,chemotherapy,tipo_qt_neo_her2,radioterapia,hormone_therapy_yn,seguim_qt_realizou,seguim_qt,data_2,weight,height
0,54,NaN,NaN,1948-03-28,2.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01,3585.0,2.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1,54,registro_de_tumores,2.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,1.0,2011-03-10,1338.0,1.0,1.0,0.0,50,NaN,NaN,NaN,9.0,509.0,3.0,MAMA SOE (EXCLUI PELE DA MAMA C44.5),85003.0,CARCINOMA DUCTAL INFILTRANTE SOE,10.0,11.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
2,54,registro_de_tumores,3.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,445.0,3.0,PELE DO TRONCO,88211.0,FIBROMATOSE AGRESSIVA,99.0,99.0,99.0,99.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
3,54,dados_histopatologicos_mama,2.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,80,80,1.0,5.0,10.0,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
4,54,dados_histopatologicos_mama,3.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,3.0,3.0,3.0,NaN,NaN,NaN,1.0,1.0,1.0,2.0,100,80,1.0,5.0,20.0,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN


# 2.0 - Manipulação dos dados

## 2.1 - Premissas de priorização, escolha de colunas de maior relevância e etiquetamento

Com o intuito de facilitar a construção de um modelo preditivo com assertividade relevante, algumas colunas de dados foram priorizadas em detrimento de outras. A premissa utilizada para a escolha dessas colunas baseia-se em dois principais motivos:

1. A relevância conhecida dos dados (o quanto as informações presentes nas colunas são importantes para a evolução da doença); e
2. A expressividade dos dados (quantas tuplas foram, para determinada coluna, preenchidas de maneira satisfatória).

Com base nisso, e com o objetivo de promover a compreensão total dos dados, foram realizados tratamentos de etiquetamento e dicionarização das colunas. Por etiquetamento entende-se modificações e ajustes nos nomes das colunas. Dito isso, exibe-se abaixo a relação das features adotadas e seus produtos:

| Nome da coluna          | Etiquetamento                          | Conteúdo                                                                                                     | Justificativa                                                                                                                     |   |
|-------------------------|----------------------------------------|--------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------|---|
| record_id               | patient_id                             | número de identificação                                                                                      | é necessário distinguir uma paciente de outra                                                                                     |   |
| dob                     | date_of_birth                          | data de nascimento                                                                                           | pode ser útil para identificar padrões relativos à idade                                                                          |   |
| race                    | race                                   | raça                                                                                                         | pode ser útil para distinguir uma possível reação diferente à doença                                                              |   |
| pregnancy_history       | has_children                           | histórico de gravidez (se teve filhos ou não)                                                                | pode ser útil para entender a influência ou não da gravidez sobre o câncer                                                        |   |
| partos                  | number_of_births                       | quantidade de partos concebidos                                                                              | é necessário para avaliar o impacto da quantidade de partos e a progressão da doença                                              |   |
| abortion                | abortion                               | histórico abortivo (se sofreu abortos ou não)                                                                | é necessário para avaliar o impacto entre abortos e a progressão do câncer                                                        |   |
| breast_feeding          | has_breastfed                          | histórico de amamentação (se já amamentou ou não)                                                            | pode ser útil para entender o impacto da amamentação e a progressão da doença                                                     |   |
| breast_feeding_time     | breastfed_time                         | período de tempo pelo qual amamentou (em meses)                                                              | é necessário para entender a influência do tempo de amamentação sobre o desenvolvimento do câncer                                 |   |
| menarche                | menarche_age                           | menarca (idade, em anos, da primeira menstruação)                                                            | é necessário para entender a influência da idade da primeira menstrução sobre o desenvolvimento da doença                         |   |
| period                  | menopause_status                       | status da menopausa                                                                                          | pode ser útil para entender se existe alguma relação entre o status da menopausa e o desenvolvimento do câncer                    |   |
| bmi                     | body_mass_index                        | IMC medido (índice de massa corporal)                                                                        | pode ser útil para entender se existe alguma relação entre o índice de massa corporal e o desenvolvimento da doença               |   |
| hormone_therapy         | hormone_therapy_status                 | histórico de terapia hormonal (se fez terapia de reposição hormonal ou não)                                  | é necessário para entender a influência da terapia de reposição hormonal sobre o desenvolvimento do câncer                        |   |
| tempo_rep_hormo         | hormone_therapy_time                   | tempo de terapia de reposição hormonal (em anos)                                                             | pode ser útil para entender se existe alguma relação entre o tempo de terapia de reposição hormonal e o desenvolvimento da doença |   |
| antec_fam_cancer_mama   | has_breast_cancer_family_history       | histórico de câncer de mama na familia (se tem parentes com câncer ou não)                                   | pode ser útil para entender o impacto genético sobre o desenvolvimento do câncer                                                  |   |
| tobaco                  | tobaco                                 | histórico de fumo (se fez o uso de tabaco ou não)                                                            | é necessário para entender a influência do consumo de tabaco sobre o desenvolvimento da doença                                    |   |
| alcohol                 | alcohol                                | histórico de bebidas alcóolicas (se fez uso de bebida alcóolica ou não)                                      | é necessário para entender a influência do consumo de álcool sobre o desenvolvimento do câncer                                    |   |
| benign                  | is_benign                              | tipo do tumor (se é benigno ou maligno)                                                                      | é necessário para entender a correlação entre pessoas com tumores benignos ou malignos e o desenvolvimento da doença              |   |
| primary_diganosis       | primary_diagnostic                     | diagnóstico primário (tipo histológico)                                                                      | é necessário para entender a correlação entre o primeiro diagnótico dado e o desenvolvimento do câncer                            |   |
| diff_tubular            | tubular_differentiation                | diferenciação tubular ou grau arquitetural                                                                   | é necessário para entender características biomorfológicas do câncer                                                              |   |
| mitotic_index           | mitotic_index                          | índice mitótico (medida da rapidez com que as células cancerígenas crescem e se dividem).                    | é necessário para entender características biomorfológicas do câncer                                                              |   |
| histological_grade      | histological_grade                     | grau histológico (o quão diferente a arquitetura do tecido tumoral está em relação ao tecido mamário normal) |  é necessário para entender características biomorfológicas do câncer                                                             |   |
| grau_hist               | histological_grade_based_on_nottinghan | grau histológico baseado em Nottinghan                                                                       |  é necessário para entender características biomorfológicas do câncer                                                             |   |
| tumor_subtype           | tumor_subtype                          | subtipo tumoral                                                                                              | é necessário para entender a correlação existente entre o subtipo do câncer a evolução da doença                                  |   |
| progesteron_perct     | progesteron_percentage                 | receptor de progesterona (quantificação  em %)                                                               | é necessário para entender a correlação entre os índices de receptores de progesterona no corpo e a evolução do câncer            |   |
| date_last_fu            | date_of_last_patient_information       | data da última informação sobre o paciente                                                                   | pode ser útil para entender a validade da última informação do paciente conhecida                                                 |   |
| follow_up_days_recidive | days_since_the_last_tumor              | tempo desde o diagnóstico até a primeira recidiva                                                            | pode ser útil para estabelecer a correlação entre o diagnóstico e status do cancer com uma possível recidiva                      |   |
| ultinfo                 | last_known_patient_status              | última informação do paciente                                                                                | é necessário para caracterizar o último estado conhecido do paciente                                                              |   |
| recidive                | ricidive_status                        | histórico de recidiva (se sofreu recidiva ou não)                                                            | é necessário para estabelecer a correlação entre uma possível recidiva e a evolução da doença                                     |   |
| dtrecidiva              | recidive_date                          | data de recidiva (se houver)                                                                                 | é necessário para estebelecer um período médio de tempo até uma possível recidiva                                                 |   |
| follow_up_days_recidive | days_until_recidive                    | tempo desde o diagnóstico até a primeira recidiva (em dias)                                                  | é necessário para estebelecer um período médio de tempo até uma possível recidiva                                                 |   |
| data_2                  | exam_date                              | data da consulta                                                                                             | é necessário para criar correlações baseadas em tempo da evolução do câncer                                                       |   |
| weight                  | weight                                 | peso                                                                                                         | pode ser útil para estabelecer possíveis correlações entre o peso e a evolução da doença                                          |   |
| height                  | height                                 | altura                                                                                                       | pode ser útil para estabelecer possíveis correlações entre a altura e a evolução do câncer                                        |   |

## 2.2 - Selecionando apenas as colunas desejadas

Dadas as colunas expostas no tópico anterior, cria-se um conjunto de dados com apenas as features desejadas, tal que:

In [ ]:
# Sobrescrevendo a tabela com apenas as colunas desejadas.
table = table[['record_id', 'dob', 'race','pregnancy_history','partos','abortion','breast_feeding','breast_feeding_time','menarche','period','bmi','hormone_therapy','tempo_rep_hormo','antec_fam_cancer_mama','tobaco','alcohol','benign','primary_diganosis','diff_tubular','mitotic_index','histological_grade','grau_hist','tumor_subtype','progesteron_perct','date_last_fu','follow_up_days_recidive','ultinfo','recidive','dtrecidiva','follow_up_days_recidive','data_2','weight','height']].copy()

# Renomeando as colunas de acordo com o etiquetamento feito.
table = table.rename(columns={'record_id': 'patient_id',
                              'dob': 'date_of_birth', 
                              'race':'race',
                              'pregnancy_history':'has_children',
                              'partos':'number_of_births',
                              'abortion':'abortion',
                              'breast_feeding':'has_breastfed',
                              'breast_feeding_time':'breastfed_time',
                              'menarche':'menarche_age',
                              'period':'menopause_status',
                              'bmi':'body_mass_index',
                              'hormone_therapy':'hormone_therapy_status',
                              'tempo_rep_hormo':'hormone_therapy_time',
                              'antec_fam_cancer_mama':'has_breast_cancer_family_history',
                              'tobaco':'tobaco',
                              'alcohol':'alcohol',
                              'benign':'is_benign',
                              'primary_diganosis':'primary_diagnostic',
                              'diff_tubular':'tubular_differentiation',
                              'mitotic_index':'mitotic_index',
                              'histological_grade':'histological_grade',
                              'grau_hist':'histological_grade_based_on_nottinghan',
                              'tumor_subtype':'tumor_subtype',
                              'progesteron_perct':'progesteron_percentage',
                              'date_last_fu':'date_of_last_patient_information',
                              'follow_up_days_recidive':'days_since_the_last_tumor',
                              'ultinfo':'last_known_patient_status',
                              'recidive':'ricidive_status',
                              'dtrecidiva':'recidive_date',
                              'follow_up_days_recidive':'days_until_recidive',
                              'data_2':'exam_date',
                              'weight':'weight',
                              'height':'height'})


In [ ]:
# Imprimindo os cinco primeiros registros com os nomes das coluna atualizados.
table.head()

,patient_id,date_of_birth,race,has_children,number_of_births,abortion,has_breastfed,breastfed_time,menarche_age,menopause_status,body_mass_index,hormone_therapy_status,hormone_therapy_time,has_breast_cancer_family_history,tobaco,alcohol,is_benign,primary_diagnostic,tubular_differentiation,mitotic_index,histological_grade,histological_grade_based_on_nottinghan,tumor_subtype,progesteron_percentage,date_of_last_patient_information,days_until_recidive,last_known_patient_status,ricidive_status,recidive_date,days_until_recidive,exam_date,weight,height
0,54,1948-03-28,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01,NaN,2.0,NaN,NaT,NaN,NaT,NaN,NaN
1,54,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1338.0,NaN,1.0,2011-03-10,1338.0,NaT,NaN,NaN
2,54,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN
3,54,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,NaN,1.0,80,NaT,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN
4,54,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,3.0,3.0,NaN,2.0,100,NaT,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN


## 2.3 Criando uma lista com pacientes únicos

Dado o escopo da aplicação, faz-se necessário, visando realizar manipulações futuras, por meio de números de identificações (ID's), obter uma lista com todos os pacientes únicos:



In [ ]:
# Criando um vetor com todos os números de identificação únicos de cada paciente.
unique_patient_ids = table['patient_id'].unique();

# Imprimindo a lista gerada.
unique_patient_ids

array([   54,   302,   710, ..., 82122, 82205, 82240])

## 2.4 Replicando valores encontrados a fim de eliminar nulos no caso de preenchimento parcial

A configuração inicial do banco de dados e a maneira pelo qual ele foi preenchido exige que os registros sejam duplicados ao longo das tuplas de um mesmo paciente. Dessa forma é possível estender a disponibilidade dos dados e permitir as manipulações necessárias à criação do algoritmo.

In [ ]:
# Realizando um loop para cada id que temos na lista unique_patient_ids
for id in unique_patient_ids:
  
  # Criando um conjunto de dados com todos os registros que possuem um mesmo identificador (em outras palavras, criando uma tabela com os registros de um único paciente somente).
  current_data_set = table.loc[table["patient_id"] == id ]
    
  # Alocando espaço na memória para armazenar um futuro valor do paciente para o dado da coluna corrente.
  value = None

  # Realizando um loop para cada linha da coluna corrente.
  for line in current_data_set.index:

    # Verificando se há a existência de um valor do paciente para o dado da coluna corrente na linha atual.
    if( not pd.isnull(current_data_set['last_known_patient_status'][line]) ):

      # Se há existência de um valor, salvando-o no cache.
      value = current_data_set['last_known_patient_status'][line]

    # Caso nao exista um valor, definindo o valor da célula como o valor salvo em cache.  
    else:

      current_data_set['last_known_patient_status'][line] = value
  
  # Adicionando o conjunto de dados atual ao conjunto de dados modificado.
  table.loc[table['patient_id'] == id, 'last_known_patient_status'] = value

table

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,patient_id,date_of_birth,race,has_children,number_of_births,abortion,has_breastfed,breastfed_time,menarche_age,menopause_status,body_mass_index,hormone_therapy_status,hormone_therapy_time,has_breast_cancer_family_history,tobaco,alcohol,is_benign,primary_diagnostic,tubular_differentiation,mitotic_index,histological_grade,histological_grade_based_on_nottinghan,tumor_subtype,progesteron_percentage,date_of_last_patient_information,days_until_recidive,last_known_patient_status,ricidive_status,recidive_date,days_until_recidive,exam_date,weight,height
0,54,1948-03-28,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01,NaN,2.0,NaN,NaT,NaN,NaT,NaN,NaN
1,54,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1338.0,2.0,1.0,2011-03-10,1338.0,NaT,NaN,NaN
2,54,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2.0,NaN,NaT,NaN,NaT,NaN,NaN
3,54,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,NaN,1.0,80,NaT,NaN,2.0,NaN,NaT,NaN,NaT,NaN,NaN
4,54,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,3.0,3.0,NaN,2.0,100,NaT,NaN,2.0,NaN,NaT,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63417,82240,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-13,NaN,2.0,NaN,NaT,NaN,NaT,NaN,NaN
63418,82240,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2.0,NaN,NaT,NaN,NaT,NaN,NaN
63419,82240,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaT,NaN,2.0,NaN,NaT,NaN,NaT,NaN,NaN
63420,82240,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2.0,NaN,NaT,NaN,2020-12-08,80.35,161.0


## 2.5 Instanciando a idade em anos
Levando em consideração o conhecimento que se tem até então sobre o câncer de mama, acredita-se, fortemente, que a idade possui impacto relevante no seu desenvolvimento e evolução, de tal forma que se torna necessário o acesso a ela, preferivelmente, em anos, tal que:

In [ ]:
# Criando uma coluna para armazenar a idade em anos.
table['age_when_cancer_was_discovered_in_years'] = np.nan

# Realizando um loop para cada paciente único existente.
for id in unique_patient_ids:

  # Criando uma tabela temporária para apenas um paciente.
  id_table = table.loc[table["patient_id"] == id]

  # Pegando o valor da data de nascimento e excluindo valores nulos.
  date_of_birth_id = id_table.date_of_birth.values
  date_of_birth_id = [x for x in date_of_birth_id if pd.isnull(x) == False]

  # Pegando as datas de exames e excluindo valores nulos.
  data_id = id_table.exam_date.values
  data_id = [x for x in data_id if pd.isnull(x) == False]

  if data_id != [] and date_of_birth_id != []:

    # Pegando apenas um valor da lista para a data de nascimento
    date_of_birth_id = date_of_birth_id[0]

    # Pegando a primeira data de consulta
    min_date = min(data_id)

    # Achando o número de dias entre a data de nascimento e a primeira consulta
    date_delta = min_date - date_of_birth_id
    date_delta = date_delta / np.timedelta64(1, 'D')

    # Associando o paciente à sua idade quando o câncer foi descoberto em anos.
    table.loc[table['patient_id'] == id, 'age_when_cancer_was_discovered_in_years'] = int(date_delta/365)

In [ ]:
table.head()

,patient_id,date_of_birth,race,has_children,number_of_births,abortion,has_breastfed,breastfed_time,menarche_age,menopause_status,body_mass_index,hormone_therapy_status,hormone_therapy_time,has_breast_cancer_family_history,tobaco,alcohol,is_benign,primary_diagnostic,tubular_differentiation,mitotic_index,histological_grade,histological_grade_based_on_nottinghan,tumor_subtype,progesteron_percentage,date_of_last_patient_information,days_until_recidive,last_known_patient_status,ricidive_status,recidive_date,days_until_recidive,exam_date,weight,height,age_when_cancer_was_discovered_in_years
0,54,1948-03-28,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01,NaN,2.0,NaN,NaT,NaN,NaT,NaN,NaN,60.0
1,54,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1338.0,2.0,1.0,2011-03-10,1338.0,NaT,NaN,NaN,60.0
2,54,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2.0,NaN,NaT,NaN,NaT,NaN,NaN,60.0
3,54,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,NaN,1.0,80,NaT,NaN,2.0,NaN,NaT,NaN,NaT,NaN,NaN,60.0
4,54,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,3.0,3.0,NaN,2.0,100,NaT,NaN,2.0,NaN,NaT,NaN,NaT,NaN,NaN,60.0


## 2.6 Criando uma coluna com o tempo de tratamento
Achamos que o tempo que a pessoa ficou tratando o câncer também seja algo importante para o modelo considerar. Criamos então uma coluna que vai guardar o número de dias entre a primeira consulta e a data de óbito

In [ ]:
# Criando uma coluna chamada time_with_tumor_delta
table['time_with_tumor_delta'] = np.nan

for id in unique_patient_ids:

  # Criando uma tabela temporária para apenas um paciente
  id_table = table.loc[table['patient_id'] == id]

  # Pegando as datas de exame e tirando valores nulos
  data_id = id_table.exam_date.values
  data_id = [x for x in data_id if pd.isnull(x) == False]

  if data_id != []:

    # Pegando a primeira data de consulta
    min_date = min(data_id)

    #Pegando a data de óbito
    date_last_fu = id_table.date_of_last_patient_information.values
    date_last_fu = [x for x in date_last_fu if pd.isnull(x) == False]
    if date_last_fu != []:
      max_date = max(date_last_fu)

      # Achando o número de dias entre a primeira consulta e a data de óbito
      date_delta = max_date - min_date
      date_delta = date_delta / np.timedelta64(1, 'D')

      # Colocando em todas as linhas do paciente
      userindex = table.index[table['patient_id'] == id]

      for index in userindex:
        table.loc[userindex, ["time_with_tumor_delta"]] = date_delta

## 2.7 Capturando os dados de pacientes que já faleceram em decorrência da doença

Com o fim de realizar uma análise primária sobre o problema, instancia-se uma tabela com somente os pacientes que já faleceram de câncer:


In [ ]:
# Criando uma nova tabela com apenas os pacientes que morreram de câncer.
table_dead_tumor = table.loc[table['last_known_patient_status'] == 3]

# Criando uma lista com os id's únicos de pacientes que morreram de câncer.
unique_records_ids = table_dead_tumor["patient_id"].unique()


## 2.X Passos futuros

Dado o construído até o momento, deseja-se, agora, elaborar os tratamentos finais com o fim de possibilitar o treinamento do algoritmo. São os próximos passos a serem tomados: transformação de dados alfanuméricos em numéricos e inserção de colunas relevantes ao projeto dadas àquelas já existentes.